# RAG Tools: Database Setup

## 1. Introduction

Welcome to the second notebook in our RAG Tools series! In this notebook, we'll set up Docker containers for our PostgreSQL database with pgvector extension and Neo4j graph database. We'll ensure both databases have accessible browser interfaces for easy management.

By the end of this notebook, you'll have:
1. Set up environment variables for database configuration
2. Created a Docker Compose configuration for our databases
3. Implemented utility classes for managing configurations and Docker containers
4. Launched and verified our database containers

## 2. Databases - PgVector and Neo4j

In our RAG (Retrieval-Augmented Generation) system, we're using a combination of PgVector (PostgreSQL with vector extensions) and Neo4j. Let's understand why:

1. **PgVector (PostgreSQL with vector extensions)**:
   - Allows us to store and efficiently query high-dimensional vectors
   - Crucial for embedding-based search and similarity comparisons in machine learning applications
   - Provides fast similarity search capabilities, essential for our RAG system

2. **Neo4j (Graph Database)**:
   - Excels at representing and querying complex relationships between entities
   - Allows us to model and query interconnected data more naturally than in traditional relational databases
   - Useful for tasks like knowledge graph construction and traversal

By combining these databases, we create a system that understands both semantic similarity (through vector embeddings in PgVector) and complex relationships (through graph structures in Neo4j). This combination is particularly powerful for tasks like code analysis, where we need to understand both the content of code (embedded into vectors) and the relationships between different code elements (represented as a graph).

## 3. Environment Configuration

Let's set up our environment variables by creating a `.env` file:

1. Navigate to the `config/` directory in your project root.
2. Create a new file named `.env` in this directory.
3. Open the `.env` file in your preferred text editor.
4. Copy and paste the following content into the file:

```
# PostgreSQL Configuration
POSTGRES_DB=ragtools_db
POSTGRES_USER=ragtools_user
POSTGRES_PASSWORD=secure_postgres_password
POSTGRES_HOST=localhost
POSTGRES_PORT=5432

# Neo4j Configuration
NEO4J_AUTH=neo4j/secure_neo4j_password
NEO4J_HOST=localhost
NEO4J_HTTP_PORT=7474
NEO4J_BOLT_PORT=7687

# Docker Configuration
POSTGRES_CONTAINER_NAME=ragtools_postgres
NEO4J_CONTAINER_NAME=ragtools_neo4j
DOCKER_NETWORK_NAME=ragtools_network
```

5. Save the file.

**IMPORTANT**: Remember to update the passwords in this file with secure values before proceeding.

The `.env` file plays a crucial role in our framework as a centralized repository for environment-specific configuration variables. As we progress through our project, we'll continually add new variables to this file, allowing us to easily manage and update our configuration settings without modifying our code.

## 4. Docker Compose Configuration

Now, let's create our `docker-compose.yml` file:

1. In the same `config/` directory, create a new file named `docker-compose.yml`.
2. Open the `docker-compose.yml` file in your text editor.
3. Copy and paste the following content into the file:

```yaml
version: '3.8'

services:
  postgres:
    image: ankane/pgvector
    container_name: ${POSTGRES_CONTAINER_NAME}
    environment:
      POSTGRES_DB: ${POSTGRES_DB}
      POSTGRES_USER: ${POSTGRES_USER}
      POSTGRES_PASSWORD: ${POSTGRES_PASSWORD}
    ports:
      - "${POSTGRES_PORT}:5432"
    volumes:
      - ../db_data/postgres:/var/lib/postgresql/data
    networks:
      - ragtools_network

  neo4j:
    image: neo4j:latest
    container_name: ${NEO4J_CONTAINER_NAME}
    environment:
      NEO4J_AUTH: ${NEO4J_AUTH}
    ports:
      - "${NEO4J_HTTP_PORT}:7474"
      - "${NEO4J_BOLT_PORT}:7687"
    volumes:
      - ../db_data/neo4j:/data
    networks:
      - ragtools_network

networks:
  ragtools_network:
    name: ${DOCKER_NETWORK_NAME}

volumes:
  postgres_data:
  neo4j_data:
```

4. Save the file.

This Docker Compose configuration sets up two services:
1. A PostgreSQL service with the pgvector extension
2. A Neo4j service

Both services are configured to use volumes for data persistence, ensuring that your data remains intact even if the containers are stopped or removed.

## 5. Configuration Utility

To manage our configuration variables more efficiently, we'll create a `Config` class in a file named `config_utils.py`. Run the following code to create this file:

In [ ]:
import os

config_utils_path = os.path.join('..', 'src', 'utils', 'config_utils.py')

# Content of the config_utils.py file
config_utils_content = '''
import os
from dotenv import load_dotenv

class Config:
    def __init__(self):
        print(f"Current working directory: {os.getcwd()}")
        
        # Use an absolute path to the .env file
        project_root = os.path.abspath(os.path.join(os.path.dirname(__file__), '..', '..'))
        env_path = os.path.join(project_root, 'config', '.env')
        print(f"Looking for .env file at: {env_path}")
        
        if os.path.exists(env_path):
            print(f".env file found at {env_path}")
            load_dotenv(env_path)
            print("Environment variables after loading .env:")
            for key, value in os.environ.items():
                if key.startswith(('POSTGRES_', 'NEO4J_', 'DOCKER_')):
                    print(f"{key}: {value}")
        else:
            print(f".env file not found at {env_path}")
        
        # Database configurations
        self.POSTGRES_DB = os.getenv('POSTGRES_DB')
        self.POSTGRES_USER = os.getenv('POSTGRES_USER')
        self.POSTGRES_PASSWORD = os.getenv('POSTGRES_PASSWORD')
        self.POSTGRES_HOST = os.getenv('POSTGRES_HOST')
        self.POSTGRES_PORT = os.getenv('POSTGRES_PORT')
        
        self.NEO4J_AUTH = os.getenv('NEO4J_AUTH')
        self.NEO4J_HOST = os.getenv('NEO4J_HOST')
        self.NEO4J_HTTP_PORT = os.getenv('NEO4J_HTTP_PORT')
        self.NEO4J_BOLT_PORT = os.getenv('NEO4J_BOLT_PORT')
        
        # Docker configurations
        self.POSTGRES_CONTAINER_NAME = os.getenv('POSTGRES_CONTAINER_NAME')
        self.NEO4J_CONTAINER_NAME = os.getenv('NEO4J_CONTAINER_NAME')
        self.DOCKER_NETWORK_NAME = os.getenv('DOCKER_NETWORK_NAME')
        
        print("Loaded configurations:")
        for attr, value in self.__dict__.items():
            print(f"{attr}: {value}")

    def get_postgres_connection_params(self):
        return {
            "dbname": self.POSTGRES_DB,
            "user": self.POSTGRES_USER,
            "password": self.POSTGRES_PASSWORD,
            "host": self.POSTGRES_HOST,
            "port": self.POSTGRES_PORT
        }

    def get_neo4j_connection_params(self):
        return {
            "uri": f"bolt://{self.NEO4J_HOST}:{self.NEO4J_BOLT_PORT}",
            "auth": tuple(self.NEO4J_AUTH.split('/')) if self.NEO4J_AUTH else None
        }
'''

# Write the content to the config_utils.py file
with open(config_utils_path, 'w') as f:
    f.write(config_utils_content)

print(f"Updated config_utils.py file created at: {config_utils_path}")


## 6. Create DockerComposeManager

To manage our Docker environment more efficiently, let's create a Python utility called DockerComposeManager. This class will help us start, stop, and check the status of our Docker containers.

First, let's create the file:



In [ ]:
import os

# Get the current working directory
current_dir = os.getcwd()

# Construct the path to the src/utils directory
utils_dir = os.path.join(current_dir, '..', 'src', 'utils')

# Ensure the utils directory exists
os.makedirs(utils_dir, exist_ok=True)

# Construct the full path for the DockerComposeManager.py file
docker_compose_manager_path = os.path.join(utils_dir, 'DockerComposeManager.py')

# Content of the DockerComposeManager.py file
docker_compose_manager_content = """
import subprocess
import os
from dotenv import load_dotenv

class DockerComposeManager:
    def __init__(self, compose_file_path):
        self.compose_file_path = os.path.abspath(compose_file_path)
        load_dotenv(dotenv_path=os.path.join(os.path.dirname(self.compose_file_path), '.env'))

    def run_command(self, command):
        try:
            result = subprocess.run(
                f"docker compose -f {self.compose_file_path} {command}",
                shell=True, check=True, capture_output=True, text=True
            )
            print(result.stdout)
        except subprocess.CalledProcessError as e:
            print(f"Error executing command: {e}")
            print(e.stderr)

    def start_containers(self):
        self.run_command("up -d")

    def stop_containers(self):
        self.run_command("down")

    def show_container_status(self):
        self.run_command("ps")
"""

# Write the content to the DockerComposeManager.py file
with open(docker_compose_manager_path, 'w') as f:
    f.write(docker_compose_manager_content)

print(f"DockerComposeManager.py file created at: {docker_compose_manager_path}")


This `DockerComposeManager` class provides methods to start and stop containers, as well as check their status. It uses the `subprocess` module to run Docker Compose commands.

## 7. Directory Structure for Persistent Databases

To ensure data persistence for our databases, we need to create a specific directory structure. Here's how to set it up manually:

1. In your project root directory, create a new directory called `db_data`.
2. Inside the `db_data` directory, create two subdirectories:
   - `postgres`
   - `neo4j`

Your directory structure should now look like this:

```
RAG_tools/
├── config/
│   ├── docker-compose.yml
│   └── .env
├── db_data/
│   ├── postgres/
│   └── neo4j/
├── notebooks/
├── src/
│   └── utils/
│       ├── config_utils.py
│       └── DockerComposeManager.py
└── tests/
```

This structure ensures that your database data will be stored persistently on your host machine, even when Docker containers are stopped or removed.

## 8. Launch Docker Containers

Now that we have our configuration utility and DockerComposeManager, let's use them to launch our Docker containers:

In [ ]:
import os
import sys
import time
import docker

# Add the project root directory to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(project_root)

from src.utils.DockerComposeManager import DockerComposeManager
from src.utils.config_utils import Config

# Load configuration
config = Config()

# Create an instance of DockerComposeManager
docker_compose_path = os.path.join(project_root, 'config', 'docker-compose.yml')
docker_manager = DockerComposeManager(docker_compose_path)

# Start the containers
print("Starting Docker containers...")
docker_manager.start_containers()

# Wait for a few seconds to allow containers to fully start
time.sleep(10)

# Check the status of the containers
print("\nChecking container status:")
docker_manager.show_container_status()

# Verify that all expected containers are running
client = docker.from_env()
expected_containers = [config.POSTGRES_CONTAINER_NAME, config.NEO4J_CONTAINER_NAME]
all_running = True

for container_name in expected_containers:
    try:
        containers = client.containers.list(filters={'name': container_name})
        if containers:
            container = containers[0]
            if container.status == 'running':
                print(f"{container_name} is running.")
            else:
                print(f"{container_name} is not running. Status: {container.status}")
                all_running = False
        else:
            print(f"{container_name} not found.")
            all_running = False
    except docker.errors.APIError as e:
        print(f"Error checking container {container_name}: {e}")
        all_running = False

if all_running:
    print("\nAll containers are running successfully!")
else:
    print("\nSome containers are not running. Please check the logs for more information.")

# Print connection information
print("\nConnection Information:")
print(f"PostgreSQL: {config.POSTGRES_HOST}:{config.POSTGRES_PORT}")
print(f"Neo4j (HTTP): {config.NEO4J_HOST}:{config.NEO4J_HTTP_PORT}")
print(f"Neo4j (Bolt): {config.NEO4J_HOST}:{config.NEO4J_BOLT_PORT}")


## 9. Verify Container Status and Database Connections

Finally, let's verify our container status and test our database connections:

In [ ]:
import os
import sys
import psycopg2
from neo4j import GraphDatabase
import time
from dotenv import load_dotenv

# Add the project root directory to the Python path
notebook_path = os.getcwd()
project_root = os.path.abspath(os.path.join(notebook_path, '..'))
sys.path.append(project_root)

print(f"Project root: {project_root}")
print(f"Python path: {sys.path}")

from src.utils.config_utils import Config

def verify_database_connections():
    config = Config()

    def wait_for_postgres(max_attempts=5, delay=5):
        for attempt in range(max_attempts):
            try:
                conn_params = config.get_postgres_connection_params()
                print(f"Attempting to connect to PostgreSQL with params: {conn_params}")
                conn = psycopg2.connect(**conn_params)
                conn.close()
                print("Successfully connected to PostgreSQL")
                return True
            except psycopg2.OperationalError as e:
                print(f"Attempt {attempt + 1}/{max_attempts}: PostgreSQL is not ready yet. Error: {e}. Retrying in {delay} seconds...")
                time.sleep(delay)
        return False

    def wait_for_neo4j(max_attempts=5, delay=5):
        for attempt in range(max_attempts):
            try:
                conn_params = config.get_neo4j_connection_params()
                print(f"Attempting to connect to Neo4j with params: {conn_params}")
                driver = GraphDatabase.driver(**conn_params)
                with driver.session() as session:
                    result = session.run("RETURN 1 AS x")
                    assert result.single()['x'] == 1
                driver.close()
                print("Successfully connected to Neo4j")
                return True
            except Exception as e:
                print(f"Attempt {attempt + 1}/{max_attempts}: Neo4j is not ready yet. Error: {e}. Retrying in {delay} seconds...")
                time.sleep(delay)
        return False

    postgres_ready = wait_for_postgres()
    neo4j_ready = wait_for_neo4j()

    if postgres_ready and neo4j_ready:
        print("\nAll database connections are successful!")
    else:
        print("\nSome database connections failed. Please check your configuration and container logs.")

    if postgres_ready:
        try:
            conn = psycopg2.connect(**config.get_postgres_connection_params())
            cur = conn.cursor()
            cur.execute("SELECT * FROM pg_available_extensions WHERE name = 'vector';")
            result = cur.fetchone()
            if result:
                print("pgvector extension is available in PostgreSQL")
            else:
                print("pgvector extension is not available. Please make sure it's installed correctly.")
            cur.close()
            conn.close()
        except Exception as e:
            print(f"Error checking pgvector extension: {e}")

    print("\nConnection Information:")
    print(f"PostgreSQL: {config.POSTGRES_HOST}:{config.POSTGRES_PORT}")
    print(f"Neo4j (HTTP): http://{config.NEO4J_HOST}:{config.NEO4J_HTTP_PORT}")
    print(f"Neo4j (Bolt): bolt://{config.NEO4J_HOST}:{config.NEO4J_BOLT_PORT}")

verify_database_connections()
